In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
#imports

import time
from io import StringIO
import torch
import numpy as np
import pandas as pd
import random
from openai import OpenAI
from sqlalchemy import create_engine
from google.colab import drive, userdata
import gradio as gr
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# Model Constants
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# Authentication

hf_token = userdata.get("HF_TOKEN")
openai_api_key = userdata.get("OPENAI_API_KEY")
if not hf_token or not openai_api_key:
 raise ValueError("Missing HF_TOKEN or OPENAI_API_KEY. Set them as environment variables.")

login(hf_token, add_to_git_credential=True)
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Tokenizer Setup

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# Model Quantization for Performance Optimization

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
# Load Model Efficiency

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
def generate_ev_driver(num_records, address_type):
    # Adjusting the prompt based on checkbox selection
    address_prompts = {
        "international": f"Generate {num_records} rows of synthetic personal data with international addresses and phone numbers.",
        "us_only": f"Generate {num_records} rows of synthetic personal data with U.S.-only addresses and phone numbers.",
        "us_international": f"Generate {num_records} rows of synthetic personal data with a mix of U.S. and international addresses and phone numbers.",
        "americas": f"Generate {num_records} rows of synthetic personal data with a mix of U.S., Canada, Central America, and South America addresses and phone numbers.",
        "europe": f"Generate {num_records} rows of synthetic personal data with Europe-only addresses and phone numbers.",
    }

    address_prompt = address_prompts.get(address_type, "Generate synthetic personal data.")
    # Generate unique driver IDs
    driver_ids = random.sample(range(1, 1000001), num_records)

    user_prompt = f"""
    {address_prompt}
    Each row should include:
    - driverid (unique from the provided list: {driver_ids})
    - first_name (string)
    - last_name (string)
    - email (string)
    - phone_number (string)
    - address (string)
    - city (string)
    - state (string)
    - zip_code (string)
    - country (string)

    Ensure the CSV format is valid, with proper headers and comma separation.
    """

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates structured CSV data."},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Call the new function to clean and extract the CSV data
    return clean_and_extract_csv(response)

In [ ]:
def clean_and_extract_csv(response):
    # Clean up the response and remove the last occurrence of the code block formatting
    csv_data = response.choices[0].message.content.strip()
    csv_data = csv_data.rsplit("```", 1)[0].strip()

    # Define header and split the content to extract the data
    header = "driverid,first_name,last_name,email,phone_number,address,city,state,zip_code,country"
    _, *content = csv_data.split(header, 1)

    # Return the cleaned CSV data along with the header
    return header + content[0].split("\n\n")[0] if content else csv_data

In [ ]:
def update_dataset(num_records, address_type):
    response = generate_ev_driver(num_records, address_type)

    # Convert response to DataFrame
    try:
        df = pd.read_csv(StringIO(response))
    except Exception as e:
        return pd.DataFrame(), f"Error parsing dataset: {str(e)}"

    return df, response

In [ ]:
# Function to handle address type selection
def check_address_selection(selected_type):
    if not selected_type:
        # Return the error message and set button to yellow and disabled
        return (
            "<span style='color:red;'>⚠️ Address type is required. Please select one.</span>",
            gr.update(interactive=False, elem_classes="yellow_btn")
        )
    # Return success message and set button to blue and enabled
    return (
        "<span style='color:green;'>Ready to generate dataset.</span>",
        gr.update(interactive=True, elem_classes="blue_btn")
    )


In [ ]:
# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("## Dynamic CSV Dataset Viewer")

    num_records_slider = gr.Slider(minimum=5, maximum=50, step=5, value=20, label="Number of Records")

    with gr.Row(equal_height=True):
        address_type_radio = gr.Radio(
            ["us_only", "international", "us_international", "americas", "europe"],
            value="",
            label="Address and Phone Type",
            info="Select the type of addresses and phone numbers"
        )
        status_text = gr.Markdown(
            "<span style='color:red;'>⚠️ Please select an address type above to proceed.</span>",
            elem_id="status_text"
        )

    generate_btn = gr.Button("Generate Data", interactive=True, elem_id="generate_btn")

    response_text = gr.Textbox(value="", label="Generated Driver List CSV", interactive=False)
    dataframe_output = gr.Dataframe(value=pd.DataFrame(), label="Generated Driver List Dataset")

    # Update status text and button style dynamically
    address_type_radio.change(fn=check_address_selection, inputs=[address_type_radio], outputs=[status_text, generate_btn])

    generate_btn.click(update_dataset, inputs=[num_records_slider, address_type_radio], outputs=[dataframe_output, response_text])

    # Custom CSS for button colors
    app.css = """
    .blue_btn {
        background-color: green;
        color: white;
    }
    """

app.launch(share=True)  # Ensure sharing is enabled in Colab

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2d1cc8e9fab628c1ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
